**From:** Raj, Database Administration Team Lead

**To:** Cloud Team 

**Subject:** Action Requested: Retrieve data from old database 

Hello,

We are looking to get some information from one of our old databases.

You will have five hours to complete this assignment after you begin. If you do not finish within five hours, you can request more time from the team, but you will need to restart the assignment from the beginning.

1. Click **Start Lab** in the top right of the screen. (It may take up to five minutes for your lab to start. Once your lab is ready, the modal dialog will indicate so.)
2. Click **AWS** in the top left of the screen to gain access to AWS. (There should be a green circle next to it.)

If you are prompted about the new AWS console home, either version can be chosen.

3. Enter "RDS" in the search bar in the top left of the screen, and select **RDS** from the search results.
4. Select **Databases** from the left navigation and confirm you have no other instances running. If you do contact your teaching team.

Our legacy data for this client is stored as a snapshot.
5. Select **Snapshots** from the left navigation.
6. Click the "Public" tab and search for and select `arn:aws:rds:us-east-1:449978885201:snapshot:dvdrental`.
7. Click **Actions** in the top right corner and select **Restore snapshot** from the drop-down menu.

### It is very important that you follow the directions below for creating your database! 

8. Confirm the DB engine is "PostgresSQL" under "DB instance settings." (You should already know how to use postgress according to your resume (aka Coursera module).)
9. Select **Single DB instance** under "Availability and durability."
10. Enter `Project-3013` in the DB instance identifier field under "Settings." 
11. Select **Yes** for "Public access" under "Connectivity."
12. Select **Burstable classes (includes t classes)** under "DB instance class." Make sure "db.t3.micro" is selected in the drop-down menu.
13. Expand "Additional configuration" at the bottom of the page and ensure all checkboxes are unchecked.
14. Click **Restore DB instance**.

It may take around ten minutes for your instance to start. While you are waiting, enter "EC2" in the search bar in the top left of the screen, and select **EC2** from the search results.

15. Select **Security Groups** under "Network & Security" from the left navigation.

You should see one labeled "default." This was just created by your new RDS instance.

16. Check the box for "default".
17. Click **Actions** at the top and select **Edit inbound rules** from the drop-down menu.
18. Click **Add rule** and select **PostgresSQL** from the drop-down menu under "Type."
19. Select **Anywhere-IPv4** from the drop-down menu under "Source" and click **Save rules**. 

Navigate back to the RDS console and check on your database instance under "Databases."

Once your database shows a status of "Available," continue the steps below.

20. Click "project-3013" and click **Modify** in the top right corner. 
21. Generate a random set of number and letters at least ten characters long and enter it in the "New master password" field under "Settings." Re-enter the password in the "Confirm password" field.

**Note**: It is important that you keep this password for later.

22. Scroll down and click **Continue**.
23. Select **Apply immediately** and click **Modify DB instance**.

Wait for your instance to be Available again - this may take up to three minutes.

In [4]:
## Lets import our requirements
import boto3
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [5]:
## This is found under "Connectivity & security" on your instance's database page:
endpoint = 'project-3013.c8rvyecrzlpn.us-east-1.rds.amazonaws.com'

# This is the default admin/root account. Leave this value as 'postgres':
user = 'postgres'

# This is the master password you set above:
password = 'Ghu45!shdb87'

## Do not change this value or rename this variable, it is used by the autograder below:
connection = create_engine(f'postgresql+psycopg2://{user}:{password}@{endpoint}/dvdrental', pool_recycle=3600);

In [6]:
# Does this connection work? 
pd.read_sql("select * from film limit 5",con=connection)

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,133,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,2006,1,7,4.99,117,14.99,NC-17,2013-05-26 14:50:58.951,[Trailers],'chamber':1 'fate':4 'husband':11 'italian':2 ...
1,384,Grosse Wonderful,A Epic Drama of a Cat And a Explorer who must ...,2006,1,5,4.99,49,19.99,R,2013-05-26 14:50:58.951,[Behind the Scenes],'australia':18 'cat':8 'drama':5 'epic':4 'exp...
2,8,Airport Pollock,A Epic Tale of a Moose And a Girl who must Con...,2006,1,6,4.99,54,15.99,R,2013-05-26 14:50:58.951,[Trailers],'airport':1 'ancient':18 'confront':14 'epic':...
3,98,Bright Encounters,A Fateful Yarn of a Lumberjack And a Feminist ...,2006,1,4,4.99,73,12.99,PG-13,2013-05-26 14:50:58.951,[Trailers],'boat':20 'bright':1 'conquer':14 'encount':2 ...
4,1,Academy Dinosaur,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,2013-05-26 14:50:58.951,"[Deleted Scenes, Behind the Scenes]",'academi':1 'battl':15 'canadian':20 'dinosaur...


You can find the Entity Relationship Diagram below:
![erd](dvd.png)

My team was asked to retrieve information on some high-paying customers.
Can you use your SQL skills to create a new database table that contains the following information?

`customer first and last name, total payments made`

**Note**: `customer first and last name` is a single field.

For example:

|John Smith| 201.23|
|Sanjay Srivastava| 10.13|

This should be limited to the top 10 customers sorted by total amount paid.

Rather than writing SQL code, you should use read_sql and to_sql to create your table.

**This must be saved to a new table titled "cust_top" with columns "full_name" and "total_amount"**

---

*We encourage you to try to construct the query on your own based on your knowledge from previous classes, but since this is not the focus of this class you have the option to reveal the query by creating a cell in this notebook with these contents:*

```
from query_answer import QUERY
print(QUERY)
```

---

In [17]:
###
### YOUR CODE HERE
###

cust_top = pd.read_sql('''SELECT
                            CONCAT(MAX(customer.first_name), ' ', MAX(customer.last_name)) AS full_name,
                            SUM(payment.amount) AS total_amount
                            FROM customer
                            JOIN payment ON payment.customer_id = customer.customer_id
                            GROUP BY customer.customer_id
                            ORDER BY SUM(payment.amount) DESC
                            LIMIT 10''', con = connection)

#cust_top.to_sql('cust_top', con = connection)

In [16]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [11]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
